# Wind speed estimation

This is small example demonstrates the usage of `estimate_ws_with_floris` function to estimate the wind speed at a given point using the FLORIS model. 

In this example the wind speed is estimated from artificial power data generated using the FLORIS model.  The data includes a 1 m/s bias to the original wind speed which is corrected via the estimator

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from floris import FlorisModel, TimeSeries

from flasc import FlascDataFrame
from flasc.utilities.floris_tools import estimate_ws_with_floris

### Establish biased wind speed data

In [ ]:
file_path = Path.cwd()
fm_path = file_path / "../floris_input_artificial/gch.yaml"
fm = FlorisModel(fm_path)

# Set to 1 turbine layout with a linear sweep over wind speeds
N = 25
wind_speeds = np.linspace(0.01, 20.0, N)
time_series = TimeSeries(
    wind_speeds=wind_speeds, wind_directions=270.0, turbulence_intensities=0.06
)
fm.set(layout_x=[0], layout_y=[0], wind_data=time_series)
fm.run()

# Construct df_scada from the FLORIS output
df_scada = FlascDataFrame(
    {
        "time": np.arange(0, N),
        "pow_000": fm.get_turbine_powers().squeeze() / 1000.0,
        "ws_000": wind_speeds + 1.0,  # Add 1m/s bias
    }
)
print(df_scada.head())

### Run wind speed estimation procedure

In [ ]:
df_scada = estimate_ws_with_floris(df_scada, fm)
print(df_scada.head())

### Calculate power with wind speed and estimated wind speed

In [ ]:
# Calculate with biased measurement
time_series = TimeSeries(
    wind_speeds=df_scada.ws_000.values, wind_directions=270.0, turbulence_intensities=0.06
)
fm.set(wind_data=time_series)
fm.run()
power_from_original_ws = fm.get_turbine_powers().squeeze() / 1000.0

# Calculate with estimated wind speed
time_series = TimeSeries(
    wind_speeds=df_scada.ws_est_000.values, wind_directions=270.0, turbulence_intensities=0.06
)
fm.set(wind_data=time_series)
fm.run()
power_from_estimated_ws = fm.get_turbine_powers().squeeze() / 1000.0

# Compute the error of each relative to measured power
original_ws_error = df_scada.pow_000.values - power_from_original_ws
estimated_ws_error = df_scada.pow_000.values - power_from_estimated_ws

# Plot the error against the measured power
fig, ax = plt.subplots()
sns.scatterplot(x=df_scada.pow_000, y=original_ws_error, ax=ax, label="Original WS", s=100)
sns.scatterplot(x=df_scada.pow_000, y=estimated_ws_error, ax=ax, label="Estimated WS")
ax.set_xlabel("Measured Power [kW]")
ax.set_ylabel("Error [kW]")
ax.set_title("Error vs Measured Power")
ax.grid(True)
plt.show()